Limpieza de datos Rooms

In [20]:
import pandas as pd
import numpy as np
import re 

In [428]:
data_location = "./Data/properati.csv"
data = pd.read_csv(data_location, sep=",")
print(data.dtypes)
data.head(3)

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


In [498]:
data["rooms_completo"] =data["rooms"]
data_room_null_description = data.description.astype(str).str.lower()
display(data_room_null_description)

0         2 ambientes tipo casa planta baja por pasillo,...
1         venta de departamento en décimo piso al frente...
2         2 ambientes 3er piso lateral living comedor am...
3         ph 3 ambientes con patio. hay 3 deptos en lote...
4         departamento con fantástica iluminación natura...
                                ...                        
121215    torre forum alcorta - máxima categoría.impecab...
121216    excelente e impecable casa en venta en las lom...
121217    venta departamento ambiente divisible a estren...
121218    2 amb al contrafrente,  luminoso. el departame...
121219    departamento de 77 m2 apto profesional. edific...
Name: description, Length: 121220, dtype: object

In [499]:
print("Primer grupo de filas sin rooms")
data["rooms_completo"].isnull().value_counts()

Primer grupo de filas sin rooms


True     73830
False    47390
Name: rooms_completo, dtype: int64

Seteamos los que son monoambientes
Primero buscamos las palabras que identifiquen al monoambiente. 
Luego buscamos las coincidencias dentro del universo de coincidencias del campo descripción
donde hubo coincidencias, le asignamos 1. 

In [500]:
def fun_buscar_patt(data_validar,patt_room):
    patt_regex = re.compile(patt_room)
    data_match = data_validar.apply(lambda x:  x if x is np.NaN else patt_regex.search(x))
    return data_match

In [501]:
data_room_match = fun_buscar_patt(data_room_null_description,r"(mono ambiente|monoambiente|mono-ambiente)")
data_room_match_mask = data_room_match.notnull()
data_room_match[data_room_match_mask]=0
data_room_mask = data["rooms_completo"].isnull()

data.loc[data_room_match_mask,"rooms_completo"] = data_room_match[data_room_match_mask].apply(lambda x: x if x is np.NaN else x)
print("Primera disminución Monoambientes")
display(data["rooms_completo"].isnull().value_counts())

Primera disminución Monoambientes


True     69080
False    52140
Name: rooms_completo, dtype: int64

In [502]:
data_encontrada= fun_buscar_patt(data_room_null_description,r"([1-9]{1,3}) *(habitaci)")

encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
room_null_mask = data["rooms_completo"].isnull()

data.loc[room_null_mask,"rooms_completo"] = encontradas[room_null_mask].apply(lambda x: x if x is np.NaN else x)
print("segunda disminución habitaciones")
data["rooms_completo"].isnull().value_counts()

segunda disminución habitaciones


True     67503
False    53717
Name: rooms_completo, dtype: int64

In [503]:
data_encontrada= fun_buscar_patt(data_room_null_description,r"([1-9]{1,3}) *(dormitorio|domitorio)")

encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
room_null_mask = data["rooms_completo"].isnull()

data.loc[room_null_mask,"rooms_completo"] = encontradas[room_null_mask].apply(lambda x: x if x is np.NaN else x)
print("segunda disminución dormitorio")
data["rooms_completo"].isnull().value_counts()

segunda disminución dormitorio


False    77068
True     44152
Name: rooms_completo, dtype: int64

In [504]:
data_encontrada= fun_buscar_patt(data_room_null_description,r"(\w+) *(dormitorio|domitorio)")

encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
room_null_mask = data["rooms_completo"].isnull()
numcol_cat_mapper = {
   'un': 1, 'uno': 1,'dos': 2,'tres': 3,'cuatro':4,
    'cinco':5,'sinco':5,'seis':6,'ceis':6,'siete':7,
    'ciete':7,'ocho':8,'nueve':9,'diez':10,'once':11,
    'doce':12,'trece':13,'siete':14,'quince':15,
    'dieciseis':16,'diecisiete':17,'dieciocho':18,'diecinueve':19,
    'veinte':20
}
encontradas1 = encontradas.map(numcol_cat_mapper)

data.loc[room_null_mask,"rooms_completo"]= encontradas1[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)))
print("Disminución en función a los ambientes escritos en texto")
data["rooms_completo"].isnull().value_counts()

Disminución en función a los ambientes escritos en texto


False    88577
True     32643
Name: rooms_completo, dtype: int64

Ahora vamos con los que pertenecen a varios ambientes, se parte de la premisa que 2 ambientes tiene al menos una habitación y sucesivamente. 

In [505]:
data_encontrada= fun_buscar_patt(data_room_null_description,r"([1-9]{1,3}) *(ambiente)")

encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
room_null_mask = data["rooms_completo"].isnull()

data.loc[room_null_mask,"rooms_completo"]= encontradas[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)-1))
print("Disminución en función a los ambientes")
data["rooms_completo"].isnull().value_counts()

Disminución en función a los ambientes


False    98427
True     22793
Name: rooms_completo, dtype: int64

In [506]:
data_encontrada= fun_buscar_patt(data_room_null_description,r"(\w+) *(ambiente)")

encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
room_null_mask = data["rooms_completo"].isnull()
numcol_cat_mapper = {
   'un': 1, 'uno': 1,'dos': 2,'tres': 3,'cuatro':4,
    'cinco':5,'sinco':5,'seis':6,'ceis':6,'siete':7,
    'ciete':7,'ocho':8,'nueve':9,'diez':10,'once':11,
    'doce':12,'trece':13,'siete':14,'quince':15
}
encontradas1 = encontradas.map(numcol_cat_mapper)

data.loc[room_null_mask,"rooms_completo"]= encontradas1[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)-1))
print("Disminución en función a los ambientes escritos en texto")
data["rooms_completo"].isnull().value_counts()

Disminución en función a los ambientes escritos en texto


False    100080
True      21140
Name: rooms_completo, dtype: int64

data_encontrada= fun_buscar_patt(data_room_null_description,r"([1-9]{1}) *(amb)$")

encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
room_null_mask = data["rooms_completo"].isnull()

data.loc[room_null_mask,"rooms_completo"]= encontradas[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)-1))
print("Disminución en función a los ambientes tipo: Amb")
data["rooms_completo"].isnull().value_counts()

In [509]:
data_encontrada= fun_buscar_patt(data_room_null_description,r"([1-9]{1}) *(amb)")

encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
room_null_mask = data["rooms_completo"].isnull()
data.loc[room_null_mask,"rooms_completo"]= encontradas[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)-1))
print("Disminución en función a los ambientes tipo: Amb")
data["rooms_completo"].isnull().value_counts()

Disminución en función a los ambientes tipo: Amb


False    101738
True      19482
Name: rooms_completo, dtype: int64

In [534]:
data_encontrada= fun_buscar_patt(data_room_null_description,r"([1-9]{1}) *(dorm)")

encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
room_null_mask = data["rooms_completo"].isnull()
data.loc[room_null_mask,"rooms_completo"]= encontradas[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)-1))
print("Disminución en función a los ambientes tipo: dorm")
data["rooms_completo"].isnull().value_counts()

Disminución en función a los ambientes tipo: dorm


False    102377
True      18843
Name: rooms_completo, dtype: int64

In [551]:
data_room_search = data["description"][data["rooms_completo"].isnull()]


In [556]:
data_encontrada= fun_buscar_patt(data_room_search,r"(dormitorio + dormitorio)")
data_encontrada
# encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.search())
# encontradas_mask = encontradas.isnull()
# encontradas[encontradas_mask]
# encontradas

1        NaN
27       NaN
47       NaN
49       NaN
57       NaN
          ..
121157   NaN
121160   NaN
121209   NaN
121214   NaN
121219   NaN
Name: description, Length: 18843, dtype: float64

In [554]:
#data.loc[room_null_mask,"rooms_completo"]= encontradas[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)-1))
# print("Disminución en función a los ambientes tipo: dorm")
# data["rooms_completo"].isnull().value_counts()
encontradas[74958]

KeyError: 74958

In [525]:
cita = "What one programmer cando in one month, two programmers can do in two months."
ocurrencias = cita.count("can")
ocurrencias


2

In [553]:
#76874 = 2
idreg= 74958
print(data.title[idreg])
display(data.loc[idreg:idreg,["rooms_completo","description"]])
data_room_null_description[idreg]

VENDE:DEPARTAMENTO


,rooms_completo,description
74958,NaN,UBICADO EN LA CALLE MANZANA 9 B “6” 2º PISO...


'ubicado en la calle manzana  9  b “6”  2º piso “d” "barrio don orione" (claypole)superficie de 104m2 incluido la cocheraliving de 3m x 8,80mbalcon de 3m x 1,65mcocina y lavadero de 3m x 4mbaño de 1,50m x 2,50m con hidromasajebaño de 1,50m x 2,50 dormitorio de 3,40m x 2,95m  con placarddormitorio de 3,40m x 2,95m  con placarddormitorio de 2,92m  x 2,50   con placardcochera doble con escritura valor: $820.000 l/gtoneguzzo inmobiliariae.de burzaco 601 . burzacotel: /8855'

In [459]:


data.loc[idreg:idreg+1,:]

Departamento - Pilar


,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,rooms_completo
88305,88305,sell,apartment,Pilar,|Argentina|Bs.As. G.B.A. Zona Norte|Pilar|,Argentina,Bs.As. G.B.A. Zona Norte,3429979.0,NaN,NaN,...,1353.121951,1981.357143,NaN,NaN,NaN,http://www.properati.com.ar/1b73m_venta_depart...,Le Bleu Negocios InmobiliariosVirrey Olaguer y...,Departamento - Pilar,https://thumbs4.properati.com/0/9Oq7nPpkXmbNjo...,NaN
88306,88306,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1b73p_venta_depart...,El precio se compone de un monto en U$S que co...,BE LIBERTADOR - SEGUNDA ETAPA,https://thumbs4.properati.com/4/wSiOd7hgFO047h...,NaN
